In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.sparse import csr_matrix
import sklearn
import itertools
import scipy as sp
from scipy import spatial
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import ndcg_score

In [44]:
#read data 
data = pd.read_csv("movie_recommendation.csv")
pd.set_option('display.max_columns', None)
movies = pd.read_csv("data/movies2.csv")
test_data = pd.read_csv("recommendation_test_data.csv")

## movie based

In [32]:
def prepare_data():
    """
    function to normalize the ratings
    :return:dataframe with normalized ratings
    """
    #storing the users ids and movie ids and teh movies ratings into dataframe
    data2 = data[["userId", "movieId", "rating"]]
    #grouping data by the userid and sum ratings for each users and store it into dataframe
    group1 = data2.groupby(['userId'],as_index = False, sort=False)["rating"].sum()
    #merge the dataframe that has the sum with the uses_data dataframe
    data2 = pd.merge(data2, group1, on = "userId", how = "left", sort = False)
    #grouping data by user id and count the ratings
    group2 = data2.groupby(['userId'],as_index = False, sort=False)["rating_x"].count()
    #merging the users_data with the grouupby dataframe
    data2 = pd.merge(data2, group2, on = "userId", how = "left", sort = False)
    #normalize the each rate and store it in a new column
    data2["normalize_rating"] = (data2["rating_x_x"] - data2["rating_y"]) / data2["rating_x_y"]
    #rename some columns in the data
    data_normalize = data2.rename(columns = {"rating_x_x":"rating", "rating_y":"sum", "rating_x_y":"count"})
    #drop some unuseful columns
    data_normalize = data_normalize.drop(columns = ["rating", "sum", "count"])
    
    return data_normalize

In [33]:
def user_sim_matrix():
    """
    generate similarity matrix between the users
    :return: matrix that has the similarities between all the users 
    """
    #calling prepare data function to normalize rating
    data_normalize=prepare_data()
    # generate a matrix of moveids as an index and userid as columns and the values is the normalize rating
    user_similarity_mat = data_normalize.pivot(index='movieId', columns='userId', values='normalize_rating').fillna(0)
    #reseting the index 
    user_similarity_mat.reset_index(inplace=True)
    #drop movie id column
    user_similarity_mat = user_similarity_mat.drop('movieId', axis=1)
    #genrate data frame from the matrix
    user_ratings = pd.DataFrame(user_similarity_mat, index = user_similarity_mat.columns, columns = user_similarity_mat.columns)
    #calculate the cosine similarity by the user ratings
    similarity = cosine_similarity(user_ratings)
    #generate a data frame from the similarity between users
    user_ratings = pd.DataFrame(similarity, index = user_ratings.index, columns = user_ratings.index)

    return user_ratings

In [34]:
def top_users(user):
    """function that give the most similar users and get their data
    return:dataframe contains all the data related to the similar users 
    """
    #calling function user_sim_matrix return teh similarity matrix between all users
    user_ratings=user_sim_matrix()
    
    users = []
    #sort the data frame and take teh most 10 similar users
    sim_values = user_ratings.sort_values(by=user, ascending=False).loc[:,user].tolist()[1:11]
    sim_users = user_ratings.sort_values(by=user, ascending=False).index[1:11]
    #put the similar user adn their similarity in a tuple
    zipped = zip(sim_users, sim_values,)
    #add the user id to a list  
    for user, sim in zipped:
        users.append(user)
    #initialize a dataframe
    sim_user_data = pd.DataFrame()
    users.append(249)
    #store normalize ratings in datafram 
    data_normalize=prepare_data()
    #iterate through the users list to return a datafrae containe the normalize rating for each similar user
    for user in users:
        sim_user_data = sim_user_data.append(data_normalize[data_normalize["userId"] == user])
    return sim_user_data

In [35]:
def movie_sim():
    """to fill the zeros in the matrix
    return:dataframe that has all the mising values filled
    """
    #storing the matrix that has all the data about the similier users 
    sim_user_data=top_users(249)
    #create a matrix of the user as index and teh movies as columns and normalized ratings as values
    movie_similarity_mat = sim_user_data.pivot(index='userId', columns='movieId', values='normalize_rating')
    #get the mean for the unwatched movies 
    df = movie_similarity_mat.mean() 
    #iterate through columns in movie_similarity_mat matrix and fill the zeros in it with teh values in the values in teh df matrix
    for column in movie_similarity_mat.columns:
        movie_similarity_mat[column] = movie_similarity_mat[column].fillna(df[column]) 
    return movie_similarity_mat,df

In [36]:
#calling the move_sim function to get the fina version of the filled matrix
movie_similarity_mat,df = movie_sim()

In [37]:
movie_similarity_mat

movieId    1         2         3         5         6         7         8       \
userId                                                                          
43      -4.508772 -3.028570 -4.508772 -4.508772 -3.450711 -4.508772 -4.508772   
126     -3.555436 -3.028570 -3.521716 -3.959226 -3.450711 -4.338802 -4.508772   
176     -3.555436 -3.028570 -3.521716 -3.959226 -3.450711 -4.338802 -4.508772   
217     -2.755302 -2.758564 -2.760196 -3.959226 -2.758564 -4.338802 -4.508772   
249     -3.692639 -3.692639 -3.521716 -3.959226 -3.450711 -4.338802 -4.508772   
253     -3.555436 -3.028570 -3.521716 -3.959226 -3.450711 -4.338802 -4.508772   
307     -2.661538 -2.663077 -2.662051 -3.959226 -3.450711 -4.338802 -4.508772   
337     -4.155844 -3.028570 -4.155844 -4.155844 -4.142857 -4.168831 -4.508772   
378     -3.906250 -3.028570 -3.521716 -3.959226 -3.450711 -4.338802 -4.508772   
395     -3.555436 -3.000000 -3.521716 -3.959226 -3.450711 -4.338802 -4.508772   
509     -3.207709 -3.028570 -3.521716 -3.213062 -3.450711 -4.338802 -4.508772   

movieId    9         10        11        12       13        14        16      \
userId                                                                         
43      -3.449895 -4.517544 -4.517544 -3.462882 -3.20985 -4.168831 -3.408435   
126     -3.449895 -3.370518 -4.517544 -3.462882 -3.20985 -4.168831 -3.408435   
176     -3.449895 -3.916667 -4.517544 -3.462882 -3.20985 -4.168831 -3.408435   
217     -2.756933 -2.755302 -4.517544 -2.756933 -3.20985 -4.168831 -3.408435   
249     -3.449895 -3.370518 -4.517544 -3.462882 -3.20985 -4.168831 -3.408435   
253     -3.449895 -3.370518 -4.517544 -3.462882 -3.20985 -4.168831 -3.408435   
307     -3.449895 -2.663077 -4.517544 -3.462882 -3.20985 -4.168831 -2.661026   
337     -4.142857 -3.370518 -4.517544 -4.168831 -3.20985 -4.168831 -4.155844   
378     -3.449895 -3.370518 -4.517544 -3.462882 -3.20985 -4.168831 -3.408435   
395     -3.449895 -3.000000 -4.517544 -3.462882 -3.20985 -4.168831 -3.408435   
509     -3.449895 -3.370518 -4.517544 -3.462882 -3.20985 -4.168831 -3.408435   

movieId    17        18        19        20        21        22        23      \
userId                                                                          
43      -3.682847 -2.662564 -3.080640 -3.225841 -2.663077 -2.710261 -4.508772   
126     -3.682847 -2.662564 -3.080640 -3.225841 -2.663077 -2.710261 -4.508772   
176     -3.682847 -2.662564 -3.080640 -3.225841 -2.663077 -2.710261 -4.508772   
217     -3.682847 -2.662564 -2.760196 -2.758564 -2.663077 -2.756933 -4.508772   
249     -3.682847 -2.662564 -3.693117 -3.693117 -2.663077 -2.710261 -4.508772   
253     -3.682847 -2.662564 -3.080640 -3.225841 -2.663077 -2.710261 -4.508772   
307     -3.682847 -2.662564 -2.661538 -3.225841 -2.663077 -2.663590 -4.508772   
337     -4.155844 -2.662564 -3.080640 -3.225841 -2.663077 -2.710261 -4.508772   
378     -3.682847 -2.662564 -3.080640 -3.225841 -2.663077 -2.710261 -4.508772   
395     -3.682847 -2.662564 -3.080640 -3.225841 -2.663077 -2.710261 -4.508772   
509     -3.209850 -2.662564 -3.207709 -3.225841 -2.663077 -2.710261 -4.508772   

movieId    24        25        27        28        29        31        32      \
userId                                                                          
43      -3.403223 -4.155844 -3.402967 -3.208779 -4.508772 -3.403480 -3.429322   
126     -3.403223 -4.155844 -3.402967 -3.208779 -4.382046 -3.403480 -3.429322   
176     -3.403223 -4.155844 -3.402967 -3.208779 -4.382046 -3.403480 -3.429322   
217     -3.403223 -4.155844 -3.402967 -3.208779 -4.382046 -3.403480 -3.429322   
249     -3.403223 -4.155844 -3.402967 -3.208779 -4.382046 -3.403480 -3.691683   
253     -3.403223 -4.155844 -3.402967 -3.208779 -4.255319 -3.403480 -3.429322   
307     -2.663590 -4.155844 -2.663077 -3.208779 -4.382046 -2.664103 -2.662051   
337     -4.142857 -4.155844 -4.142857 -3.208779 -4.382046 -4.142857 -4.155844   
378     -3.403223 -4.155844 -3.402967 -3.

In [38]:
def showing_recommendation(movie_similarity_mat):
    """function that predict the movies using the similarity matrix 
    args: movie similarity matrix dataframe
    return: recommended movies titles
    """
    #take the first row of teh dataframe
    recommended_movies = movie_similarity_mat.iloc[4]
    #reseting the index for the recommended_movies dataframe
    recommended_movies = pd.DataFrame(recommended_movies).reset_index()
    #rename a column after reset the index 
    recommended_movies = recommended_movies.rename(columns = {1:'ratings'})
    #store the wtached movies for user 1 
    watched_movies = data[data["userId"] == 249]["movieId"]
    #convert the list of watched movies to dataframe
    watched_movies = pd.DataFrame(watched_movies)
    #drop the watched movies from the predicted movies
    cond = recommended_movies['movieId'].isin(watched_movies['movieId'])
    recommended_movies.drop(recommended_movies[cond].index, inplace = True)
    
    return recommended_movies

In [39]:
#call the showing_recommendation function to stor ethe recommended movies
recommended_movies = showing_recommendation(movie_similarity_mat)

recommended_movies_title = []
#convert the movies ids into titles
for Id in recommended_movies["movieId"]:
    recommended_movies_title.append(movies[movies["movieId"] == Id]["title"])
recommended_movies_title = list(itertools.chain.from_iterable(recommended_movies_title))

In [40]:
#show the top 10 recommneded movies
recommended_movies_title[0:10]

['Grumpier Old Men (1995)',
 'Father of the Bride Part II (1995)',
 'Heat (1995)',
 'Sabrina (1995)',
 'Tom and Huck (1995)',
 'Sudden Death (1995)',
 'GoldenEye (1995)',
 'American President, The (1995)',
 'Dracula: Dead and Loving It (1995)',
 'Balto (1995)']

# model-based (ALS) 

In [41]:
from pyspark.ml.recommendation import ALSModel 
from pyspark.sql import SparkSession 
from pyspark import SparkContext
import pandas as pd
import numpy as np
import itertools

# create spark object
spark = SparkSession.builder.appName('Recommendation_system').getOrCreate() 

In [42]:
# read the model
model = ALSModel.load('als_model2')

In [45]:
# convert test data to spark data frame 
test_data = spark.createDataFrame(test_data) 
# get the prediction data as spark data frame
prediction = model.transform(test_data)
# convert to pandas
prediction = prediction.toPandas() 
test_data = test_data.toPandas() 

In [46]:
def predict_movies(als_model, test_data, user_id):
    """
    predict movies id's for given user
    :Args:
    als_model (ALS model): model to predict movies
    test_data (pandas dataframe): datarame contains test data 
    user_id (int): id for given user to predict movies
    :return: list of recommended movies id's
    """
    #test_data = test_data[test_data["userId"] == uid]

    # convert pandas dataframe to pyspark dataframe
    test_data = spark.createDataFrame(test_data) 
    # predict movies fo all users using als model
    prediction = als_model.transform(test_data)
    # convert pyspark dataframe to pandas dataframe
    prediction = prediction.select("*").toPandas()
    # sort values in dataframe by prediction
    prediction = prediction.sort_values(by='prediction', ascending=False)
    # select highest predict rating and return it's movie id
    prediction = prediction[prediction["userId"] == user_id]["movieId"].head()
    return prediction 

In [47]:
recommended_movies_ids = predict_movies(model,test_data,448)

In [48]:
recommended_movies_title=[]
for Id in recommended_movies_ids:
    recommended_movies_title.append(movies[movies["movieId"] == Id]["title"])
recommended_movies_title = list(itertools.chain.from_iterable(recommended_movies_title))

In [49]:
recommended_movies_title

['Shaft (2000)',
 'Bottle Rocket (1996)',
 "Pete's Dragon (1977)",
 'Operation Dumbo Drop (1995)',
 'Scary Movie 3 (2003)']

# Evaluation 

### Evaluate Model Based

In [50]:
def normalize_data(df_rating):
    """
    reads the dataset from the same directory contains this file, clean and normalized it
    :Args:
    user_test_data (list): list contains random user as test data
    :return: dataframe ready to use in ML algorithm to train a model
    """
    #storing the users ids and movie ids and teh movies ratings into dataframe
    users_data = df_rating[["userId", "movieId", "rating"]]
    #grouping data by the userid and sum ratings for each users and store it into dataframe
    users_group1 = users_data.groupby(['userId'],as_index = False, sort=False)["rating"].sum()
    #merge the dataframe that has the sum with the uses_data dataframe
    users_data = pd.merge(users_data, users_group1, on = "userId", how = "left", sort = False)
    #grouping data by user id and count the ratings
    users_group2 = users_data.groupby(['userId'],as_index = False, sort=False)["rating_x"].count()
    #merging the users_data with the grouupby dataframe
    users_data = pd.merge(users_data, users_group2, on = "userId", how = "left", sort = False)
    #normalize the each rate and store it in a new column
    users_data["normalize_rating"] = (users_data["rating_x_x"] - users_data["rating_y"]) / users_data["rating_x_y"]
    #rename some columns in the data
    data_normalize = users_data.rename(columns = {"rating_x_x":"rating", "rating_y":"sum", "rating_x_y":"count"})
    #drop some unuseful columns
    data_normalize = data_normalize.drop(columns = ["rating", "sum", "count"])
    
    return data_normalize

In [51]:
def calculate_dcg(rate):
    
    """
    This function to calculate dcg for the list it receives 
    Args: list of rate which could be predict_rate or actual_rate
    :return: dcg for the input list 
    """
    # array to store score for each value in the list 
    dcg = []
    
    # iterate through the input list 
    for index, value in enumerate(rate): 
        # calculate numerator which is twice the value and then minus 1 of it
        numerator = 2**value - 1
        # calculate denominator by add 2 to the index because python 0-index then take log2 to it 
        denominator =  np.log2(index + 2) 
        # callculate score by dividing the numerator into denominator 
        score = numerator/denominator
        # append teh score to the dcg array
        dcg.append(score)
        
    # sum all values in the dcg list     
    DCG = sum(dcg)    
    return DCG

In [52]:
def calculate_ndcg(predict_rate, actual_rate): 
    
    """
    This function calculate the ndcg for a particular user from test users 
    Args: the prediction rating and the actual rating
    :return: average ndcg 
    """
    
    # calculate dcg for predict rating by calling calculate_dcg function 
    dcg = calculate_dcg(predict_rate)
    # calculate idcg for the actual_rate by calling calculate_dcg function 
    idcg = calculate_dcg(actual_rate)
    
    # calculate average ndcg 
    ndcg = dcg / idcg
    
    return ndcg

In [53]:
def actual_rating_test_data(test_users):
    
    """
    This function store the actual rating for the test data 
    Args: list contain users id of the test users 
    :return: data frame contain userid of test users and movieId and normalized rating 
    """
    # data frame to store data of test users 
    actual_data_test_users = pd.DataFrame()
    # iterate in the test users to get the data frame of each user
    for user in test_users:
        
        df = data[["userId","movieId","rating"]]
        df = df[df["userId"] == user]
        actual_data_test_users = actual_data_test_users.append(df)
    
    # get the normalized data
    data_normalize = normalize_data(data)
    # merge the actual_data_test_users with data_normalize to ger normalized ratinfg
    actual_data_test_users = actual_data_test_users.merge(data_normalize, how='left', left_on=['userId','movieId'], right_on = ['userId','movieId'])
    # drop the rating column from  which contain the actual rating 
    actual_data_test_users = actual_data_test_users.drop(columns= ['rating'])
    # convert the of normalize_rating column name to rating  
    actual_data_test_users = actual_data_test_users.rename(columns = {"normalize_rating":"rating"})
    
    return actual_data_test_users

In [54]:
def get_rating_test_data(test_users):
    
    """
    This function calculate the actual rating and predict rating
    Args: list contain users id of the test users 
    :return: two data frame the actual rating and the predict rating) 
    """
    
    # create data frame to get the data of test users from prediction data which contain predidction_rating column 
    pred_data_test_users = pd.DataFrame()
    # calling actual_rating_test_data function to get the data of test users from main data which contain actual_rating column 
    actual_data_test_users = actual_rating_test_data(test_users)
    
    # iterate through test users data set and append ueres data to pred_data_test_users 
    for user in test_data['userId']:
        pred_data_test_users = pred_data_test_users.append(prediction[prediction["userId"] == user])
    
    # merge pred_data_test_users with actual_data_test_users to add actual rating column in pred_data_test_users data frame 
    pred_data_test_users = pred_data_test_users.merge(actual_data_test_users, how='left', left_on=['userId','movieId'], right_on = ['userId','movieId'])

    # drop rating_x column which contain the actual rating 
    pred_data_test_users = pred_data_test_users.drop(columns = ["rating_x"])

    # convert the of rating_y column which contain normalized rating to rating column
    pred_data_test_users = pred_data_test_users.rename(columns = {"rating_y":"rating"})
    
    # get the actual rating by using groupby in each user then get his actual rating as a list 
    actual_rating = pred_data_test_users.groupby("userId", sort = False)["rating"].apply(list)
    # convert the userId from index to column in the data frame
    actual_rating = actual_rating.reset_index()
    
    # get the predict rating by using groupby in each user then get his predict rating as a list 
    predict_rating = pred_data_test_users.groupby("userId", sort = False)["prediction"].apply(list)
    # convert the userId from index to column in the data frame
    predict_rating = predict_rating.reset_index()
    
    
    return actual_rating , predict_rating

In [55]:
def compute_avg_ndcg(test_users):
    
    """
    This function to evaluate our modle using "NDCG" 
    Args: list contain users id for the test users 
    :return: average ndcg for all test users 
    """
    
    ndcg = [] # array to tore ndcg for each user in the test users 
    
    # calling predict_rating_test_data function to get the the data contain actual rating and the data contain predict rating 
    actual_rating , Predict_rating = get_rating_test_data(test_users)

    # iterate through the rating column to get ndcg result
    for row in range(len(actual_rating["rating"])): 
        
        if(len(actual_rating["rating"][row]) > 1):# check if length of actual_rating is large than one 
            
            actual_rating["rating"][row].sort(reverse=True) # sort actual_rating
            
            # convert both list of actual and predict rating to array
            actual_rate = np.asarray([actual_rating["rating"][row]]) 
            predict_rate = np.asarray([Predict_rating["prediction"][row]])
            
            # switch none value to 0 in both actual and predict rating 
            actual_rate[actual_rate == None] = 0
            predict_rate[predict_rate == None] = 0
    
            # calling calculate_ndcg function to calculate the result of the ndcg and append each value to the "ndcg" list
            ndcg.append(calculate_ndcg(predict_rate , actual_rate))

    # convert frome list of list to one list 
    list_ndcg = list(itertools.chain.from_iterable(ndcg))
    # calculate ndcg average 
    NDCG_AVG = sum(list_ndcg)/len(list_ndcg)
    
    return NDCG_AVG

In [56]:
# cslling compute_avg_ndcg function to evaluate our model 
test_users = [414, 599, 474, 448, 274, 610, 68, 380, 606, 249, 288, 387, 182, 177, 318]
NDCG_AVG = compute_avg_ndcg(test_users)
NDCG_AVG

0.9179092567162606

### Evaluate Movie Based 

In [57]:
def evaluate_movie_based(testusers):
    
    data_normalize = prepare_data()
    ndcg_scores=[]
    for user in testusers:
        user_data = data_normalize[data_normalize["userId"] == user]
        user_ids = user_data["movieId"]
        user_rating = user_data["normalize_rating"].sort_values(ascending = False)
        user_prediction = df[user_ids]
        
        user_rating = np.asarray([user_rating])
        user_prediction = np.asarray([user_prediction])
        
        #user_ndcg = ndcg_score(user_rating, user_prediction)
        #ndcg_scores.append(user_ndcg)
        avg_ndcg = calculate_ndcg(user_prediction,user_rating)
        
        
    avg_ndcg=sum(avg_ndcg)/len(avg_ndcg)
    return avg_ndcg

In [58]:
test_users=[249]

In [59]:
avg_ndcg = evaluate_movie_based(test_users)

In [60]:
avg_ndcg

0.9873108868323344